In [1]:
# !pip install opencv-python
# !pip install dlib
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=d224489dc344d62758161c1caaabd067e8722e35c2928d13e7b363abe1724d72
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyngrok
!pip install flask-ngrok
!ngrok authtoken '2ZtsQUsJm0TLjIeGPrPvazdiRgH_4pgrBSgxVsBwLX7WNW33h'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import face_recognition

RuntimeError: ignored

In [4]:
from flask import Flask, render_template, request, jsonify,send_file
from flask_ngrok import run_with_ngrok
import base64
import os
import pickle
import cv2

In [5]:
# import face_recognition


# Load known images and their corresponding names
known_faces = {

    "Prof Irshad Ahmad": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/director.jpeg"),
    "Sir Riaz Hussain": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/riaz_sir_5.jpeg"),
    "Sir Zubair": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/zubair3.jpeg"),
    "Sir Waqas Ahmad Khan": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Sir waqas.jpg"),
    "Sir Abid": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/abid.png"),
    "Sir Junaid": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/junaid_sir.jpeg"),
    "hammad": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/hammad.jpg"),
    "akhter": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/akhter.jpg"),
    "adeel": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/adeel.jpg"),
    "abbas": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/abbas.jpg"),
    "tabarak": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/tabarak.jpg"),
    "qaiser": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/qaiser.jpeg"),
    "Sir Arshad": face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/sir arshad.jpeg")



}

In [6]:
known_face_encodings = [face_recognition.face_encodings(image)[0] for name, image  in known_faces.items()]
known_face_names = [name for name, image  in known_faces.items()]

In [10]:
template_folder = '/content/drive/MyDrive/Face_recognition_project/templates'

app = Flask(__name__, template_folder=template_folder)
run_with_ngrok(app)

# Initialize variables
face_locations = []
face_names = []

def detect_faces(frame):
    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        face_names.append(name)

    return face_locations, face_names

def gen_frames():
    video_capture = cv2.VideoCapture(0)

    while True:
        success, frame = video_capture.read()
        if not success:
            break
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = detect_faces(frame)

            for (top, right, bottom, left), name in zip(face_locations, face_names):
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)

            ret, buffer = cv2.imencode('.jpg', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            if not ret:
                continue

            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index30.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/capture', methods=['POST'])
def capture():
    data_url = request.form['image']
    image_data = base64.b64decode(data_url.split(',')[1])

    # Save the captured image
    with open('captured_image.jpg', 'wb') as img_file:
        img_file.write(image_data)

    # Load and recognize the captured image
    captured_image = face_recognition.load_image_file('captured_image.jpg')
    captured_face_encodings = face_recognition.face_encodings(captured_image)

    if not captured_face_encodings:
        return jsonify(success=False, image=None, name='No face detected')

    # Compare the captured face with known faces
    matches = face_recognition.compare_faces(known_face_encodings, captured_face_encodings[0])
    if True in matches:
        first_match_index = matches.index(True)
        name = known_face_names[first_match_index]
    else:
        name = 'Unknown'

    with open('captured_image.jpg', 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode('utf-8')

    return jsonify(success=True, image=encoded_image, name=name)


@app.route("/get_audio/<file_name>", methods=['GET'])
def get_audio(file_name):
    return send_file(f"/content/drive/MyDrive/Face_recognition_project/Audio Files/{file_name}", as_attachment=True)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

students

In [ ]:
from flask import Flask, render_template, Response, request, jsonify, send_file
import cv2
import face_recognition
import base64
import os

template_folder = '/content/drive/MyDrive/Face_recognition_project/templates'

app = Flask(__name__, template_folder=template_folder)
run_with_ngrok(app)
hammad_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/hammad.jpg")
hammad_face_encoding = face_recognition.face_encodings(hammad_image)[0]

akhter_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/akhter.jpg")
akhter_face_encoding = face_recognition.face_encodings(akhter_image)[0]

adeel_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/adeel.jpg")
adeel_face_encoding = face_recognition.face_encodings(adeel_image)[0]

riaz_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Riaz sir.jpg")
riaz_face_encoding = face_recognition.face_encodings(riaz_image)[0]

zubair_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Zubair sir.jpg")
zubair_face_encoding = face_recognition.face_encodings(zubair_image)[0]

abbas_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/abbas.jpg")
abbas_face_encoding = face_recognition.face_encodings(abbas_image)[0]

tabarak_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/tabarak.jpg")
tabarak_face_encoding = face_recognition.face_encodings(tabarak_image)[0]

faisal_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/faisal.jpg")
faisal_face_encoding = face_recognition.face_encodings(faisal_image)[0]

zaki_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/sir zulqarnain.jpg")
zaki_face_encoding = face_recognition.face_encodings(zaki_image)[0]

waqas_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Sir waqas.jpg")
waqas_face_encoding = face_recognition.face_encodings(waqas_image)[0]

junaid_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Sir Junaid.jpg")
junaid_face_encoding = face_recognition.face_encodings(junaid_image)[0]

imran_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Imran sir.jpg")
imran_face_encoding = face_recognition.face_encodings(imran_image)[0]

Ibrar_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Ibrar.jpg")
Ibrar_face_encoding = face_recognition.face_encodings(Ibrar_image)[0]

abdullah_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Sir Abdullah.jpg")
abdullah_face_encoding = face_recognition.face_encodings(abdullah_image)[0]

umer_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Umer.jpg")
umer_face_encoding = face_recognition.face_encodings(umer_image)[0]

Ahmad_image = face_recognition.load_image_file("/content/drive/MyDrive/Face_recognition_project/known_faces/Ahmad.jpg")
Ahmad_face_encoding = face_recognition.face_encodings(Ahmad_image)[0]


known_face_encodings = [hammad_face_encoding, akhter_face_encoding,adeel_face_encoding,riaz_face_encoding,zubair_face_encoding,abbas_face_encoding,tabarak_face_encoding,faisal_face_encoding,zaki_face_encoding,waqas_face_encoding,junaid_face_encoding,imran_face_encoding,Ibrar_face_encoding,abdullah_face_encoding,umer_face_encoding,Ahmad_face_encoding]
known_face_names = ["hammad", "Akhter","Adeel","Riaz Sir","Zubair Sir","abbas","tabarak","faisal","Zulqarnain Sir","Waqas Sir","Junaid Sir","Imran Sir","Ibrar Junir","Sir Abdullah","Umer khan swati","Ahmad AK"]


# ... (your known_face_encodings and known_face_names definitions)

# Initialize variables
face_locations = []
face_names = []

def detect_faces(frame):
    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        face_names.append(name)

    return face_locations, face_names

def gen_frames():
    video_capture = cv2.VideoCapture(0)

    while True:
        success, frame = video_capture.read()
        if not success:
            break
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = detect_faces(frame)

            for (top, right, bottom, left), name in zip(face_locations, face_names):
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)

            # Convert the frame to base64 for displaying in the HTML
            ret, buffer = cv2.imencode('.jpg', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            if not ret:
                continue
            frame = buffer.tobytes()

            # Send the frame with recognized names to the client
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index25.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/capture', methods=['POST'])
def capture():
    data_url = request.form['image']
    image_data = base64.b64decode(data_url.split(',')[1])

    # Save the captured image
    with open('captured_image.jpg', 'wb') as img_file:
        img_file.write(image_data)

    # Load and recognize the captured image
    captured_image = face_recognition.load_image_file('captured_image.jpg')
    captured_face_encodings = face_recognition.face_encodings(captured_image)

    if not captured_face_encodings:
        return jsonify(success=False, image=None, name='No face detected')

    # Compare the captured face with known faces
    matches = face_recognition.compare_faces(known_face_encodings, captured_face_encodings[0])
    if True in matches:
        first_match_index = matches.index(True)
        name = known_face_names[first_match_index]
    else:
        name = 'Unknown'

    with open('captured_image.jpg', 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode('utf-8')

    return jsonify(success=True, image=encoded_image, name=name)

@app.route("/get_audio/<file_name>", methods=['GET'])
def get_audio(file_name):
    return send_file(f"/content/drive/MyDrive/Face_recognition_project/Audio Files/{file_name}", as_attachment=True)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://36d5-34-68-219-197.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:42:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:43:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:43:17] "POST /capture HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:43:20] "GET /get_audio/hammad.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:43:21] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:44:12] "POST /capture HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:44:15] "GET /get_audio/abbas.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:45:52] "POST /capture HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:46:02] "GET /get_audio/Akhter.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:46:18] "POST /capture HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:46:21] "GET /get_audio/tabarak.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2023 15:46:39] "